## Lựa chọn đặc trưng bằng cách xáo trộn ngẫu nhiên

Phương pháp phổ biến để lựa chọn đặc trưng gồm xáo trộn ngẫu nhiên các giá trị của một biến cụ thể và xác định hoán vị đó ảnh hưởng thế nào đến phép đo chất lượng của thuật toán học máy. Nói cách khác, ý tưởng là hoán vị các giá trị của từng đặc trưng tại thời điểm đó và đo lường mức độ hoán vị (hoặc xáo trộn các giá trị của nó) làm giảm độ chính xác hay roc_auc hoặc mse của mô hình học máy (hoặc bất kỳ phép đo chất lượng nào khác!). Nếu các biến quan trọng thì hoán vị ngẫu nhiên các giá trị sẽ giảm đáng kể bất kỳ phép đo nào trong số này. Ngược lại, hoán vị hoặc xáo trộn các giá trị sẽ ít hoặc không ảnh hưởng đến phép đo chất lượng mô hình mà chúng ta đang đánh giá.

Quy trình sẽ như sau:

- Xây dựng mô hình học máy và lưu trữ phép đo chất lượng.
- Xáo trộn 1 đặc trưng và đưa ra dự đoán mới sử dụng mô hình trước đó.
- Xác định chất lượng của dự đoán này.
- Xác định thay đổi về chất lượng của dự đoán với các đặc trưng đã xáo trộn với đặc trưng ban đầu.
- Lặp lại cho từng đặc trưng.

Để lựa chọn đặc trưng, chúng ta sẽ chọn những thứ làm giảm hiệu suất của mô hình, vượt quá ngưỡng được thiết lập tùy ý.

Chúng ta sẽ mô phỏng cách lựa chọn đặc trưng sử dụng xáo trộn ngẫu nhiên trên bài toán hồi quy và phân loại.

**Lưu ý:** Với mô phỏng, chúng ta sẽ tiếp tục sử dụng Rừng ngẫu nhiên, nhưng quy trình lựa chọn này có thể được sử dụng với thuật toán học máy. Trên thực tế, cần xác định cụ thể tầm quan trọng của các đặc trưng cho thuật toán được sử dụng. Do đó, các thuật toán khác nhau có thể trả về các tập hợp con khác nhau của các đặc trưng quan trọng. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score

## Hồi quy

In [2]:
# load tập dữ liệu
data = pd.read_csv('./datatset/house-price/houseprice.csv')
data.shape

(1460, 81)

In [3]:
# Trên thực tế, nên thực hiện lựa chọn đặc trưng sau khi tiền xử lý dữ liệu,
# tất cả các biến hạng mục được mà hóa thành số,
# và rồi chúng ta có thể đánh giá tính tất định của các mục tiêu

# để đơn giản, chúng ta sẽ chỉ sử dụng biến dạng số
# lựa chọn các cột số:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [4]:
# chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 36), (438, 36))

In [5]:
# với phương pháp này, cần thiết lập lại các index của 
# tập dữ liệu trả về

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [6]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

### Huấn luyện thuật toán học máy với tất cả các đặc trưng

In [7]:
## Yêu cầu 1
# Bước đầu tiên là xác định độ quan trọng bằng cách xáo trộn đặc trưng 
# để xây dựng mô hình học máy để 
# lựa chọn đặc trưng

# Trong trường hợp này, chúng ta sẽ cây dựng Rừng ngẫu nhiên nhưng nhớ rằng
# chúng ta có thể sử quy trình này cho thuật toán học máy khác

# xây một vài cây thấp để tránh overfitting
rf = RandomForestRegressor(n_estimators=100,
                           max_depth=3,
                           random_state=2909,
                           n_jobs=4)

## VIẾT CODE Ở ĐÂY:
rf.fit(X_train, y_train)

# in ra các số đo chất lượng
## VIẾT CODE Ở ĐÂY:
print('train rmse: ', mean_squared_error(y_train, rf.predict(X_train), squared=False))
print('train r2: ', r2_score(y_train, (rf.predict(X_train))))
print()
print('test rmse: ', mean_squared_error(y_test, rf.predict(X_test), squared=...))
print('test r2: ', r2_score(y_test, rf.predict(X_test)))

train rmse:  34125.46855017603
train r2:  0.8090829266232026

test rmse:  1533833250.8284774
test r2:  0.7740705281238518


<details><summary> Gợi ý </summary>

[mean_squared_error()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)

[r2_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)

</details>

### Xáo trộn các đặc trưng và đánh giá tình trạng chất lượng

In [8]:
## Yêu cầu 2
# Trong cell này, chúng ta sẽ xáo trộn từng đặc trưng của tập dữ liệu
#  rồi sử dụng tập dữ liệu với biến đã xáo trộn để đưa ra dự đoán
# sử dụng rừng ngẫu nhiên đã huấn luyện trong cell trước

# rmse huấn luyện tổng quát: sử dụng tất cả các đặc trưng
## VIẾT CODE Ở ĐÂY:
train_rmse = mean_squared_error(y_train, rf.predict(X_train), squared=False)

# list nắm bắt thay đổi về chất lượng
performance_shift = []

# cho từng đặc trưng:
for feature in X_train.columns:
    
    X_train_c = X_train.copy()

    # xáo trộn đặc trưng riêng lẻ
    X_train_c[feature] = X_train_c[feature].sample(frac=1, random_state=11).reset_index(
        drop=True)

    # dự đoán với đặc trưng đã xáo trộn và tính roc-auc
    ## VIẾT CODE Ở ĐÂY:
    shuff_rmse = mean_squared_error(y_train, rf.predict(X_train_c), squared=False)
    
    drift = train_rmse - shuff_rmse 

    # lưu trữ trong roc-auc
    ## VIẾT CODE Ở ĐÂY:
    performance_shift.append(drift)

In [9]:
# Bây giờ chúng ta sẽ biến đổi list thành một Series pandas
# để dễ thao tác

feature_importance = pd.Series(performance_shift)

# thêm tên biến vào index
feature_importance.index = X_train.columns

feature_importance.head()

MSSubClass         0.000000
LotFrontage      -45.751607
LotArea         -390.252300
OverallQual   -42772.508176
OverallCond       -6.967475
dtype: float64

In [10]:
# Ở đây chúng ta thấy rmse càng nhỏ càng tốt

# khi thực hiện original_rmse - shuffled_data_rmse

# nếu đặc trưng quan trọng thì dữ liệu đã xáo trộn sẽ tăng rmse

# do đó chúng ta tìm các giá trị âm ở đây

# số đặc trưng làm giảm chất lượng
# khi xáo trộn

feature_importance[feature_importance<0].shape[0]

28

In [12]:
# thêm tên biến

feature_importance[feature_importance<0].index

Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', 'ScreenPorch', 'MoSold', 'YrSold'],
      dtype='object')

### Lựa chọn đặc trưng

In [13]:
## Yêu cầu 3
# So sánh chất lượng của rừng ngẫu nhiên
# được xây chỉ sử dụng các đặc trưng đã chọn

# cắt dữ liệu
feat = feature_importance[feature_importance<0].index

## VIẾT CODE Ở ĐÂY:
X_train = X_train[feat]
X_test = X_test[feat]

In [14]:
X_train.shape, X_train.shape

((1022, 28), (1022, 28))

In [16]:
## Yêu cầu 4
# xây dựng và đánh giá mô hình

rf = RandomForestRegressor(n_estimators=100,
                           max_depth=3,
                           random_state=2909,
                           n_jobs=4)

## VIẾT CODE Ở ĐÂY:
rf.fit(X_train, y_train)

# in ra các số đo chất lượng
## VIẾT CODE Ở ĐÂY:
print('train rmse: ', mean_squared_error(y_train, rf.predict(X_train), squared=False))
print('train r2: ', r2_score(y_train, (rf.predict(X_train))))
print()
print('test rmse: ', mean_squared_error(y_test, rf.predict(X_test), squared = False))
print('test r2: ', r2_score(y_test, (rf.predict(X_test))))

train rmse:  34114.694591609696
train r2:  0.8092034587666255

test rmse:  39561.867880230304
test r2:  0.7694589242329682


Mô hình có ít đặc trưng hơn thể hiện chất lượng giống với mô hình có tất cả các đặc trưng.